In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!git clone https://github.com/tensorflow/models.git
!cd models/research; protoc object_detection/protos/*.proto --python_out=.
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pil is already the newest version (5.1.0-1).
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
fatal: destination path 'models' already exists and is not an empty directory.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.113s

OK (skipped=1)


In [4]:
!pip install object-detection

     |████████████████████████████████| 1.5MB 3.4MB/s 
     |████████████████████████████████| 337kB 50.5MB/s 
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


In [0]:
import functools
import json
import os
import sys
import tensorflow as tf

In [0]:
sys.path.append('models')
sys.path.append('models/research')
sys.path.append('models/research/object_detection')
sys.path.append('models/research/slim')

In [3]:
from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

ModuleNotFoundError: ignored

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
#Mount google drive to access the data from drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_dir = "/content/drive/My Drive/Programming/gun_detector/training/new"
num_clones = 1
clone_on_cpu = False
pipeline_config_path = "/content/drive/My Drive/Programming/gun_detector/training/ssd_inception_v2_coco_for_colab.config"

In [0]:
@tf.contrib.framework.deprecated(None, 'Use object_detection/model_main.py.')
def main():
    assert train_dir, 'train_dir is misssing'
    if pipeline_config_path:
        configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
    else:
        None
    
    model_config = configs['model']
    train_config = configs['train_config']
    input_config = configs['train_input_config']
    
    model_fn = functools.partial(model_builder.build,
                                model_config=model_config,
                                is_training=True)
    
    def get_next(config):
        return dataset_builder.make_initializable_iterator(
            dataset_builder.build(config)).get_next()
    
    create_input_dict_fn = functools.partial(get_next, input_config)
    
    env = json.loads(os.environ.get('TF_CONFIG', '{}'))
    cluster_data = env.get('cluster', None)
    cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
    task_data = env.get('task', None) or {'type': 'master', 'index': 0}
    task_info = type('TaskSpec', (object,), task_data)
    
    # Parameters for a single worker.
    ps_tasks = 0
    worker_replicas = 1
    worker_job_name = 'lonely_worker'
    task = 0
    is_chief = True
    master = ''
    
    if cluster_data and 'worker' in cluster_data:
        # Number of total worker replicas include "worker"s and the "master".
        worker_replicas = len(cluster_data['worker']) + 1
    if cluster_data and 'ps' in cluster_data:
        ps_tasks = len(cluster_data['ps'])
        
    if worker_replicas > 1 and ps_tasks < 1:
        raise ValueError('At least 1 ps task is needed for distributed training.')

    if worker_replicas >= 1 and ps_tasks > 0:
        # Set up distributed training.
        server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
        if task_info.type == 'ps':
            server.join()
            return
        
        worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
        task = task_info.index
        is_chief = (task_info.type == 'master')
        master = server.target
        
    graph_rewriter_fn = None
    
    if 'graph_rewriter_config' in configs:
        graph_rewriter_fn = graph_rewriter_builder.build(
            configs['graph_rewriter_config'], is_training=True)
        
    trainer.train(create_input_dict_fn,
                 model_fn,
                 train_config,
                 master,
                 task,
                 num_clones,
                 worker_replicas,
                 clone_on_cpu,
                 ps_tasks,
                 worker_job_name,
                 is_chief,
                 train_dir,
                 graph_hook_fn=graph_rewriter_fn)


In [0]:
main()

INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:global step 61: loss = 1.1667 (0.177 sec/step)


INFO:tensorflow:global step 61: loss = 1.1667 (0.177 sec/step)


INFO:tensorflow:global step 62: loss = 2.0754 (0.180 sec/step)


INFO:tensorflow:global step 62: loss = 2.0754 (0.180 sec/step)


KeyboardInterrupt: ignored